<a href="https://colab.research.google.com/github/abhibrat/Retrieval-Augmented-Generation/blob/main/BigBullModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install --upgrade pymilvus langchain openai tiktoken pypdf langchain-openai


In [ ]:
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.zilliz import Zilliz
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from google.colab import userdata

import os
import textwrap

# 1. Set up the name of the collection to be created.
COLLECTION_NAME = 'BigBull'

# 2. Set up the dimension of the embeddings.
DIMENSION = 1536

# 3. Set up the cohere api key
OPENAI_API_KEY = userdata.get('openai_api_key')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 4. Set up the connection parameters for your Zilliz Cloud cluster.
URI = 'https://in03-a26a5bfdf5dee2c.api.gcp-us-west1.zillizcloud.com'

# 5. Set up the token for your Zilliz Cloud cluster.
# You can either use an API key or a set of cluster username and password joined by a colon.
TOKEN = userdata.get('zilliz_token')

wrapper = textwrap.TextWrapper(width=150,
    initial_indent=" " * 4,
    subsequent_indent=" " * 4,
    break_long_words=False,
    break_on_hyphens=False)

# print(wrapper.fill(string))

In [ ]:
from langchain.schema.document import Document

# all_splits =  [Document(page_content="I am Abhi", metadata={"source": "local","title":"Identification","language":"en"})]

# loader = WebBaseLoader([
#     'https://github.com/donnemartin/system-design-primer'

# ])

# docs = loader.load()

# from bs4 import BeautifulSoup as Soup

# from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


# url = "https://raw.githubusercontent.com/donnemartin/system-design-primer/master/README.md"
# loader = RecursiveUrlLoader(
#     url=url, max_depth=6, extractor=lambda x: Soup(x, "html.parser").text
# )
# docs = loader.load()

# docs = [Document(page_content=docs[0].page_content, metadata={"source": "local","title":"System Design","language":"en"})]




In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("http://www.bseindia.com/xml-data/corpfiling/AttachHis/d0467d0f-3fee-4fe4-bbb1-eaaa5b48a2e9.pdf")

docs=[]
for doc in loader.load():
  docs.append(doc)
print(len(docs))
print(docs)

In [ ]:
# Split the documents into smaller chunks
docs=[]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
all_splits = text_splitter.split_documents(docs)
# print(docs)
embeddings = OpenAIEmbeddings()
connection_args = { 'uri': URI, 'token': TOKEN }

# vector_store = Zilliz(
#     embedding_function=embeddings,
#     connection_args=connection_args,
#     collection_name=COLLECTION_NAME,
#     drop_old=False,
#     primary_field="pk"
# ).from_documents(
#     all_splits,
#     embedding=embeddings,
#     collection_name=COLLECTION_NAME,
#     connection_args=connection_args,
# )

vector_store = Zilliz(
    embedding_function=embeddings,
    connection_args=connection_args,
    collection_name=COLLECTION_NAME,
    drop_old=False,
    primary_field="pk")
# ).from_documents(
#     all_splits,
#     embedding=embeddings,
#     collection_name=COLLECTION_NAME,
#     connection_args=connection_args,
# )

In [ ]:
query = "orchid"
docs = vector_store.similarity_search(query)

print(docs)

In [ ]:
llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)
retriever = vector_store.as_retriever()

# Use three sentences maximum and keep the answer as concise as possible.
# Always say "thanks for asking!" at the end of the answer.

template = """Use the following pieces of context to answer the question at the end. Don't give short answers.
Please give as detailed an answer as possible.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
Question: {question}
Helpful Answer:"""
rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

response=rag_chain.invoke("what are the key focus areas for ashiana housing?")


In [ ]:
# print(wrapper.fill(response.content))
print(response.content)

In [ ]:
loader = WebBaseLoader([
    'https://raw.githubusercontent.com/donnemartin/system-design-primer/master/README.md'

])

docs = loader.load()
docs

In [ ]:
from bs4 import BeautifulSoup as Soup

from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


url = "https://raw.githubusercontent.com/donnemartin/system-design-primer/master/README.md"
loader = RecursiveUrlLoader(
    url=url, max_depth=4, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()


In [ ]:
docs[0].page_content

In [ ]:
llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

# Use three sentences maximum and keep the answer as concise as possible.
# Always say "thanks for asking!" at the end of the answer.

template = """Don't give short answers.
Please give as detailed an answer as possible.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Question: {question}
Helpful Answer:"""
rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

response=rag_chain.invoke("explain with code and maths and simple example how the softmax function works")
print(wrapper.fill(response.content))
